 ### Data is from an 'Open Data Portal' (https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15094797). Data: [Financial Services Commission_Indemnity Health Insurance Information] with generated fill-ins for NaN

In [ ]:
#!pip install pandas

In [ ]:
# import requests
# import xml.etree.ElementTree as ET
# import pandas as pd
# from urllib.parse import urlencode, quote_plus
# from time import sleep

# service_key = "h1pTz4jbuiFqkFY2tRJeeMjphtXfVx%2FunhHWEHbNic%2B2Ee0Yuvg%2BfxNNWRbSsOXustnQSmLeXCvTMQRO4QMtww%3D%3D"
# base_url = "https://apis.data.go.kr/1160100/service/GetMedicalReimbursementInsuranceInfoService/getInsuranceInfo"

# all_data = []

# dates = pd.date_range(start="2021-12-01", end="2024-12-01", freq="MS")
# dates = [d.strftime("%Y%m%d") for d in dates]

# for basDt in dates:
#     page = 1
#     total_pages = 1

#     while page <= total_pages:
#         params = {
#             "serviceKey": service_key,
#             "resultType": "xml",
#             "pageNo": page,
#             "numOfRows": 100,
#             "basDt": basDt
#         }

#         url = f"{base_url}?{urlencode(params, quote_via=quote_plus)}"
#         response = requests.get(url)

#         if response.status_code != 200:
#             break

#         root = ET.fromstring(response.content)

#         header = root.find("header")
#         if header is not None and header.findtext("resultCode") != "00":
#             break

#         if page == 1:
#             total_count = int(root.findtext(".//totalCount", default="0"))
#             if total_count == 0:
#                 break
#             total_pages = (total_count // 100) + (1 if total_count % 100 > 0 else 0)

#         for item in root.iter("item"):
#             row = {child.tag: child.text for child in item}
#             row["basDt"] = basDt
#             all_data.append(row)
#         page += 1
#         sleep(0.1)


# df = pd.DataFrame(all_data)
# # df.to_csv("./private_insurance_data.csv" index=False, encoding="utf-8-sig")

In [1]:
import pandas as pd

df = pd.read_csv("./private_insurance_data.csv")
df.head()

,cmpyCd,cmpyNm,ptrn,mog,prdNm,age,mlInsRt,fmlInsRt,basDt,ofrInstNm
0,N01,삼성화재,4세대 실손의료보험,상해입원,(무)케어플랜 실손의료비보험(삼성),20,3397,2696,20220101,손해보험협회
1,N01,삼성화재,4세대 실손의료보험,상해입원,(무)케어플랜 실손의료비보험(삼성),25,3518,3942,20220101,손해보험협회
2,N01,삼성화재,4세대 실손의료보험,상해입원,(무)케어플랜 실손의료비보험(삼성),30,2812,2624,20220101,손해보험협회
3,N01,삼성화재,4세대 실손의료보험,상해입원,(무)케어플랜 실손의료비보험(삼성),35,4263,3375,20220101,손해보험협회
4,N01,삼성화재,4세대 실손의료보험,상해입원,(무)케어플랜 실손의료비보험(삼성),40,2624,3206,20220101,손해보험협회


## Comparative Study

In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv("./private_insurance_data.csv")
df["basDt"] = pd.to_numeric(df["basDt"], errors="coerce")
latest_date = df["basDt"].max()
df = df[df["basDt"] == latest_date]
df = df.dropna(subset=["mlInsRt", "fmlInsRt", "age"])
df["mlInsRt"] = pd.to_numeric(df["mlInsRt"], errors="coerce")
df["fmlInsRt"] = pd.to_numeric(df["fmlInsRt"], errors="coerce")
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df = df.dropna()
df["age_squared"] = df["age"] ** 2
df["year"] = df["basDt"] // 10000
df["log_mlInsRt"] = np.log1p(df["mlInsRt"])
df["log_fmlInsRt"] = np.log1p(df["fmlInsRt"])
categorical_cols = ["cmpyNm", "ptrn", "mog", "prdNm", "ofrInstNm"]
encoders = {col: LabelEncoder().fit(df[col]) for col in categorical_cols}
for col in categorical_cols:
    df[col] = encoders[col].transform(df[col])

features = ["cmpyNm", "ptrn", "mog", "prdNm", "ofrInstNm", "age", "age_squared", "year"]
X = df[features]

# Train model
def train_model(y_col):
    y = df[y_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=300, max_depth=15, min_samples_split=5, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mse = mean_squared_error(np.expm1(y_test), np.expm1(pred))
    print(f"{y_col} MSE: {mse:.2f}")
    return model

male_model = train_model("log_mlInsRt")
female_model = train_model("log_fmlInsRt")

decoded_df = df.copy()
for col in categorical_cols:
    decoded_df[col] = encoders[col].inverse_transform(decoded_df[col])

candidates = decoded_df[decoded_df["age"] == 30].drop_duplicates(subset=categorical_cols + ["age"])

encoded_candidates = candidates.copy()
for col in categorical_cols:
    encoded_candidates[col] = encoders[col].transform(encoded_candidates[col])

encoded_candidates["age_squared"] = encoded_candidates["age"] ** 2
encoded_candidates["year"] = latest_date // 10000

X_candidates = encoded_candidates[features]
candidates["pred_ml"] = np.expm1(male_model.predict(X_candidates))
candidates["pred_fml"] = np.expm1(female_model.predict(X_candidates))

best_male = candidates.loc[candidates["pred_ml"].idxmin()]
best_female = candidates.loc[candidates["pred_fml"].idxmin()]

print("\n✅ Best Recommendation for Male (age 30):")
print(f"Company: {best_male['cmpyNm']}, Product: {best_male['prdNm']}, Estimated Price: {best_male['pred_ml']:.2f} KRW")

print("\n✅ Best Recommendation for Female (age 30):")
print(f"Company: {best_female['cmpyNm']}, Product: {best_female['prdNm']}, Estimated Price: {best_female['pred_fml']:.2f} KRW")

log_mlInsRt MSE: 738148.44
log_fmlInsRt MSE: 680051.56

✅ Best Recommendation for Male (age 30):
Company: 현대해상, Product: (무)케어플랜 실손의료비보험(현대), Estimated Price: 1649.65 KRW

✅ Best Recommendation for Female (age 30):
Company: AIG, Product: (무)케어플랜 실손의료비보험(AI), Estimated Price: 1648.28 KRW
